In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv
from firecrawl_scraping import *
from utility import *
from llm_extraction import *
from tqdm import tqdm
import glob
import tiktoken

- gpt-4o: "o200k_base",
- gpt-4: "cl100k_base",
- gpt-3.5-turbo: "cl100k_base",
- gpt-3.5: "cl100k_base",  # Common shorthand
- gpt-35-turbo : "cl100k_base",  # Azure deployment name

gpt-4o US$5.00 / 1M input tokens； US$15.00 / 1M output tokens


In [5]:
encoding = tiktoken.encoding_for_model("gpt-4o")
print(encoding)

<Encoding 'o200k_base'>


In [6]:

def count_tokens(input_string: str) -> int:
    tokenizer = tiktoken.encoding_for_model("gpt-4o")

    tokens = tokenizer.encode(input_string)

    return len(tokens)

def calculate_cost(input_string: str, cost_per_million_tokens: float = 5) -> float:
    num_tokens = count_tokens(input_string)

    total_cost = (num_tokens / 1_000_000) * cost_per_million_tokens

    return total_cost

In [8]:
data = read_json_file('scraping_output_v2_raw/bennie.json')

data

{'/partners': 'Partner with Bennie\n===================\n\nOur partner programs are designed for technology providers, consultants, insurance carriers, and referral partners who want to connect their networks with our benefits platform. Find the program that works best for you.\n\n[Get in Touch](#become-a-partner-form)\n\n![Partners Hero](https://images.ctfassets.net/0xmico1wg6et/R5pwIIpxdPlA7rDo1Ivt5/825113b9b56b2e2d6107a195adfaf531/partners-hero_2x.png?q=85&fm=png&w=750)\n\n*   [Our Marketplace](#section-partners-2021-marketplace)\n    \n*   [Referral Partners](#section-partners-2021-referral-program)\n    \n\n![Benefits Partners Mobiles](https://images.ctfassets.net/0xmico1wg6et/7vAsQo4rbTx9594X1tbXem/60c70429fbb1cee9eed2efa8a0f928b8/benefits-partners-mobiles-v2.png?q=85&fm=png&w=695)\n\nBENEFITS Marketplace\n\nBenefits Partners\n-----------------\n\nOur benefits partners are comprised of products and services available for the Employer or Individual level. Learn more about what it 

In [13]:
print(data['/partners'])

Partner with Bennie

Our partner programs are designed for technology providers, consultants, insurance carriers, and referral partners who want to connect their networks with our benefits platform. Find the program that works best for you.

[Get in Touch](#become-a-partner-form)

![Partners Hero](https://images.ctfassets.net/0xmico1wg6et/R5pwIIpxdPlA7rDo1Ivt5/825113b9b56b2e2d6107a195adfaf531/partners-hero_2x.png?q=85&fm=png&w=750)

*   [Our Marketplace](#section-partners-2021-marketplace)
    
*   [Referral Partners](#section-partners-2021-referral-program)
    

![Benefits Partners Mobiles](https://images.ctfassets.net/0xmico1wg6et/7vAsQo4rbTx9594X1tbXem/60c70429fbb1cee9eed2efa8a0f928b8/benefits-partners-mobiles-v2.png?q=85&fm=png&w=695)

BENEFITS Marketplace

Benefits Partners
-----------------

Our benefits partners are comprised of products and services available for the Employer or Individual level. Learn more about what it takes to be listed as a Benefits Partner with Bennie.

[

In [12]:
print(clean_scraped_content(data['/partners']))

Partner with Bennie
Our partner programs are designed for technology providers, consultants, insurance carriers, and referral partners who want to connect their networks with our benefits platform. Find the program that works best for you.
[Get in Touch]
![Partners Hero]
*   [Our Marketplace]
*   [Referral Partners]
![Benefits Partners Mobiles]
BENEFITS Marketplace
Benefits Partners
Our benefits partners are comprised of products and services available for the Employer or Individual level. Learn more about what it takes to be listed as a Benefits Partner with Bennie.
[Browse Our Partners]
![Benefits Partners Image 2]
HCM Technology Marketplace
HCM Tech Partners
We believe in an open ecosystem of HCM products and data. By partnering with the Bennie platform, you can help improve the experience of our growing mutual client base and their employee populations.
[View All]
![Benefits Partners Image 3]
Expand The Bennie Network
Referral Partners
If you recommend Bennie to your network, we’ll

In [16]:
print(calculate_cost(data['main_page']))
print(calculate_cost(clean_scraped_content(data['main_page'])))

0.00941


In [11]:
for url, content in data.items():
    print(url)
    print(f'Estimated GPT4-o cost: ${calculate_cost(data[url])}')
    print(f'Estimated GPT4-o cost after cleaning: ${calculate_cost(clean_scraped_content(data[url]))}')
    print('------------------------')
    

/partners
Estimated GPT4-o cost: $0.024155000000000003
Estimated GPT4-o cost after cleaning: $0.00414
------------------------
/customers
Estimated GPT4-o cost: $0.00637
Estimated GPT4-o cost after cleaning: $0.00283
------------------------
main_page
Estimated GPT4-o cost: $0.00965
Estimated GPT4-o cost after cleaning: $0.00417
------------------------
